In [22]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import math

In [50]:
!pip install huggingface_hub
import re
import json
from sklearn.model_selection import train_test_split


data = """
Once upon a time, in a land far, far away, there lived a young princess who loved to explore the forest near her castle. She would spend hours wandering among the trees, listening to the birds sing and watching the animals play.

One day, while she was exploring a part of the forest she had never been to before, she stumbled upon a hidden clearing. In the center of the clearing was a beautiful, sparkling fountain. The princess was mesmerized by the fountain and spent hours just watching the water flow.

As the sun began to set, she realized it was time to head back to the castle. She promised herself that she would return to the clearing the next day to explore it further. Little did she know, the fountain had a magical secret that would change her life forever.

The next day, as she approached the clearing, she saw a figure standing by the fountain. It was an old woman, dressed in a cloak made of shimmering fabric. The woman smiled warmly at the princess and beckoned her closer.

"Welcome, dear child," the woman said. "I have been waiting for you. This fountain is not just any fountain; it is a magical fountain that grants wishes to those who find it. You have a pure heart, and so I will grant you one wish."

The princess was astonished. She had heard stories of magical fountains, but she never thought she would find one. She took a deep breath and made her wish. "I wish for peace and happiness for everyone in my kingdom," she said.

The old woman nodded and waved her hand over the fountain. A bright light shone from the water, and the princess felt a warm, comforting sensation wash over her. She knew that her wish had been granted.

From that day on, the kingdom flourished. The people were happy and prosperous, and the princess continued to visit the fountain, grateful for the magic that had brought so much joy to her home.
"""


import re
import json
from sklearn.model_selection import train_test_split


data_content = data.split("\n")

def build_text_files(data_text, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_text:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data_content,test_size=0.15)


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

print(train)
print(test)

Train dataset length: 14
Test dataset length: 3
['From that day on, the kingdom flourished. The people were happy and prosperous, and the princess continued to visit the fountain, grateful for the magic that had brought so much joy to her home.', 'The princess was astonished. She had heard stories of magical fountains, but she never thought she would find one. She took a deep breath and made her wish. "I wish for peace and happiness for everyone in my kingdom," she said.', '', '', 'The old woman nodded and waved her hand over the fountain. A bright light shone from the water, and the princess felt a warm, comforting sensation wash over her. She knew that her wish had been granted.', '', '"Welcome, dear child," the woman said. "I have been waiting for you. This fountain is not just any fountain; it is a magical fountain that grants wishes to those who find it. You have a pure heart, and so I will grant you one wish."', '', '', 'The next day, as she approached the clearing, she saw a fig

In [39]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [40]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=16)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=16)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [41]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 10, # Number of update steps between two evaluations.
    save_steps=20, # after # steps model is saved
    warmup_steps=50,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1731: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [42]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=3.946125030517578, metrics={'train_runtime': 49.163, 'train_samples_per_second': 1.342, 'train_steps_per_second': 0.061, 'total_flos': 538914816000.0, 'train_loss': 3.946125030517578, 'epoch': 3.0})

In [44]:
trainer.save_model()

In [49]:
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, the world was a different place.


The world was a different place.

The world was a different place.

The world was a different place.

The world was a different place.



In [52]:
from huggingface_hub import login
login()


In [53]:
model.save_pretrained('./first-gpt2-model')
tokenizer.save_pretrained('./first-gpt2-model')

('./first-gpt2-model/tokenizer_config.json',
 './first-gpt2-model/special_tokens_map.json',
 './first-gpt2-model/vocab.json',
 './first-gpt2-model/merges.txt',
 './first-gpt2-model/added_tokens.json',
 './first-gpt2-model/tokenizer.json')

In [57]:
from huggingface_hub import login, create_repo, Repository
repo_name = "fine-tuned-gpt2-colab"  # Change this to your desired repository name
from huggingface_hub import HfApi

# Initialize the HfApi instance
api = HfApi()

# Create a new repository
username = api.whoami()['name']  # Get your Hugging Face username
full_repo_name = f"{username}/{repo_name}"

# Create the repository (you can also create it on the Hugging Face website)
api.create_repo(repo_name, private=False)

api.upload_folder(
    folder_path='./first-gpt2-model',  # Path to the folder with your model
    repo_id=full_repo_name,  # Model repository name
    commit_message="Initial commit of fine-tuned GPT-2 model from Colab"
)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/metriccoders/fine-tuned-gpt2-colab/commit/1a54dd2f5ada05a6ac6de99c587c13c23b3f826e', commit_message='Initial commit of fine-tuned GPT-2 model from Colab', commit_description='', oid='1a54dd2f5ada05a6ac6de99c587c13c23b3f826e', pr_url=None, pr_revision=None, pr_num=None)